In [1]:
#import
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
from IPython.display import HTML
import re

In [2]:
#scrap data
r=requests.get('https://www.wongnai.com/_api/v1.6/cooking/recipes/tags/fruit-recipes?_v=5.132&locale=th&page.number=1&page.size=1500')
obj=json.loads(r.text)
alldata = obj['data']['page']['entities']

In [3]:
#data for dataframe
menu=[]
ing=[]
view=[]
link=[]
img=[]
for i in alldata:
    menu.append(i['title'])
    ing.append(i['ingredientKeywords'])
    view.append(i['view'])
    imgkey=i['photo']['smallUrl']
    img.append(f'<img src="{imgkey}" />')
for i in alldata:
    name=i['title']
    k=i['url']
    url=f'https://www.wongnai.com/{k}'
    linkkey=f'<a href="{url}">{name}</a>'
    link.append(linkkey)

In [4]:
#first table
table = pd.DataFrame({'menu': menu, 'ingredient': ing,
                      'view': view, 'link':link, 'photo':img})

In [5]:
#fruit data med
r=requests.get('https://medthai.com/%E0%B8%A3%E0%B8%B2%E0%B8%A2%E0%B8%8A%E0%B8%B7%E0%B9%88%E0%B8%AD%E0%B8%9C%E0%B8%A5%E0%B9%84%E0%B8%A1%E0%B9%89/')
s=BeautifulSoup(r.text, 'lxml')
#all_letter=['ก','ข','ค','ฆ','ง','จ','ฉ','ช','ซ','ฌ','ญ','ฎ','ฏ','ฐ','ฑ','ฒ','ณ','ด','ต','ถ','ท','ธ','น','บ','ป','ผ','ฝ','พ','ฟ','ภ','ม','ย','ร','ล','ว','ศ','ษ','ส','ห','ฬ','อ','ฮ']
letter=['ก','ข','ค','ง','จ','ฉ','ช','ซ','ด','ต','ถ','ท','น','บ','ป','ผ','ฝ','พ','ฟ','ม','ย','ร','ล','ว','ส','ห','อ']
mfruit_list=[]
for i in letter:
  key_letter=f'หมวด-{i}'
  d=s.find('div', {'id':key_letter})
  a_tags=d.find_all('a')
  k=a_tags
  for j in k:
    mfruit_list.append(j['href'])

mfruit =  [i.replace('/','') for i in mfruit_list]

#scrap ชื่อผลไม้ wiki
r=requests.get('https://th.wikipedia.org/wiki/%E0%B8%A3%E0%B8%B2%E0%B8%A2%E0%B8%81%E0%B8%B2%E0%B8%A3%E0%B8%9C%E0%B8%A5%E0%B9%84%E0%B8%A1%E0%B9%89')
s=BeautifulSoup(r.text, 'lxml')
d=s.find('div', {'id':'mw-content-text'})
a_tags=d.find_all('a')
tag=a_tags
wfruit_list=[]
wfruit=[]
fruit_c=[]
for i in range(20,199):
  wfruit_list.append(tag[i]['title'])
delete=['แก้ไขส่วน: ข','แก้ไขส่วน: ค','แก้ไขส่วน: ง','แก้ไขส่วน: ช','แก้ไขส่วน: ต','แก้ไขส่วน: ท','แก้ไขส่วน: น','แก้ไขส่วน: บ','แก้ไขส่วน: ฝ','แก้ไขส่วน: พ','แก้ไขส่วน: ม','แก้ไขส่วน: ร','แก้ไขส่วน: ล','แก้ไขส่วน: ส','แก้ไขส่วน: ห','แก้ไขส่วน: อ','หมวดหมู่:',' (พืช)',' (ผลไม้)']
for i in wfruit_list:
    for j in delete:
        wfruit_list=[x.replace(j,'') for x in wfruit_list]
for i in wfruit_list:
    if len(i)!=0:
        wfruit.append(i)

#combine fruit
fruit_c = list(set().union(wfruit,mfruit))

In [6]:
#edit fruit
#remove from fruit
fruit_c=[k.replace('แอปเปิ้ลไซเดอร์','แอปเปิลไซเดอร์') for k in fruit_c]
fruit_c=[k.replace('เบอร์รี่','เบอร์รี') for k in fruit_c]
for i in fruit_c:
    for j in ['สตรอเบอร์รี','สตรอเบอร์รี่','กีวี่','แอปเปิ้ล','เบอรี่','เชอร์รี่','ลูกพลับ']:
        fruit_c=[k.replace(j,'') for k in fruit_c]
#add fruit
fruit=[]
for i in fruit_c:
    if len(i)>1:
        fruit.append(i)
for j in ['พีช','เมล่อน','สตรอว์เบอร์รี','มะยงชิด','เบอร์รี']:
        fruit.append(j)

In [7]:
#rectify word in table
table_del=['น้ำตาล','น้ำตาลทราย','น้ำตาลปี๊บ','น้ำเต้าหู้','น้ำเชื่อมตาลโตนด','ปริมาณที่ให้ไว้เป็นการประมาณเท่านั้นครับ ความหวานขึ้นอยู่กับสับปะรดที่ใช้ว่าฉ่ำแค่ไหนครับ ถ้าฉ่ำมากก็แทบจะไม่ต้องเติมมะพร้าวเลย','จากสวนหลังบ้าน','จากกาแฟ','แก้วกาแฟ','สไปรูลิน่า','flaxseeds จาก superfoods',' flax seeds จาก superfoods','spirulina powder จาก superfoods','kale powder จาก superfoods','chia seeds จาก superfoods','flaxseeds จาก superfoods','acai powder จาก superfoods','maca powerder จาก superfoods']
table_straw=['สตรอว์เบอร์รี่','สตรอว์เบอรี่','สตรอว์เบอรี','สตรอเบอร์รี่','สตรอเบอร์รี','สตรอเบอรี่','สตรอเบอรี','สตอเบอรี่','สตอเบอรี','สตอเบอร์รี่','สตอเบอร์รี']
for i in table_del:
    table['ingredient']=table['ingredient'].str.replace(i,'')
for i in table_straw:
    table['ingredient']=table['ingredient'].str.replace(i,'สตรอว์เบอร์รี')
table['ingredient']=table['ingredient'].str.replace('แอปเปิ้ล','แอปเปิล').str.replace('เบอรี่','เบอร์รี').str.replace('เบอร์รี่','เบอร์รี').str.replace('เปลือกแก้งมังกร','เปลือกแก้วมังกร').str.replace('กล้วยน้ำหว้า','กล้วยน้ำว้า').str.replace('สัปปะรด','สับปะรด').str.replace('อินทผาลัม','อินทผลัม').str.replace('กรีวี่','กีวี').str.replace('กีวี่','กีวี').str.replace('แตงไท','แตงไทย').str.replace('เชอร์รี่','เชอร์รี')
table_ing=table['ingredient']
table_ing

0          ชามะลิอัญชัน, น้ำส้มทรีโอผสมเนื้อส้ม, น้ำแข็ง
1                                      แป้ง, ไข่,  ,ลำไย
2      เนื้อทุเรียน, หัวกะทิ, ทรายขาว, เกลือ, ข้าวเหน...
3                                   สตรอว์เบอร์รี, , น้ำ
4      น้ำร้อน, โกโก้, วิปครีม, ส้มโชกุนคั้นสด, น้ำแข...
                             ...                        
715    กล้วยหอม, ข้าวโอ๊ต, ไข่ไก่, นมอัลมอนด์, ผงฟู, ...
716    กล้วย, ไข่, โอ๊ตสำเร็จรูป, baking soda, นมอัลม...
717    แอปเปิลเขียว หรือแดง, แป้ง puff pastry แช่แข็ง...
718    ไข่ไก่, นม, ขนมปัง, น้ำมัน, ผลไม้ตามใจชอบ, น้ำ...
719    ทรายส่วนที่ 1, เกลือ, ผลมัลเบอร์รี, ผงเจลาติน,...
Name: ingredient, Length: 720, dtype: object

In [8]:
#search for fruit
#ing_find=table['ingredient']
ing_find=table_ing
fruit_have=[]
for i in ing_find:
    list_fruit=''
    for j in fruit:
        x=re.findall(j,i)
        if x != []:
            if list_fruit=='':
                list_fruit=j
            else:
                list_fruit=list_fruit+','+j
    if list_fruit != '':
        fruit_have.append(list_fruit)
    else:
        fruit_have.append('-')

In [9]:
#create dataframe
table = pd.DataFrame({'menu': menu, 'ingredient': table_ing,'fruit': fruit_have,
                      'view': view, 'link':link, 'photo':img})
pd.set_option('display.max_colwidth', -1)
table['ingredient'] =  table['ingredient'].apply(lambda x: re.sub(r"[^\u0E00-\u0E7Fa-zA-Z' ,]|^'|'$|''",'', str(x))) #remove emoji
HTML(table.to_html(escape=False))

<ipython-input-9-9a747f8eb597>:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


,menu,ingredient,fruit,view,link,photo
0,ชามะลิอัญชันส้ม,"ชามะลิอัญชัน, น้ำส้มทรีโอผสมเนื้อส้ม, น้ำแข็ง",ส้ม,36,ชามะลิอัญชันส้ม,
1,เค้กลำไย,"แป้ง, ไข่, ,ลำไย",ลำไย,5426,เค้กลำไย,
2,ข้าวเหนียวทุเรียน,"เนื้อทุเรียน, หัวกะทิ, ทรายขาว, เกลือ, ข้าวเหนียวซื้อสำเร็จตามร้านข้าวเหนียวไก่, ควันเทียน",ทุเรียน,37,ข้าวเหนียวทุเรียน,
3,สตรอว์เบอร์รีลอยแก้ว,"สตรอว์เบอร์รี, , น้ำ","สตรอว์เบอร์รี,เบอร์รี",2139,สตรอว์เบอร์รีลอยแก้ว,
4,โกโก้ส้มลาเต้ 🍊,"น้ำร้อน, โกโก้, วิปครีม, ส้มโชกุนคั้นสด, น้ำแข็ง, , นมสดแช่เย็น","โกโก้,ส้ม",67,โกโก้ส้มลาเต้ 🍊,
5,kiwi smoothie bowl,"กีวี, กล้วยหอม, เม็ดสน","กีวี,กล้วยหอม,กล้วย",1914,kiwi smoothie bowl,
6,กล้วยบวชชี,"กล้วยน้ำว้า, กะทิอมควันเทียน, น้ำสะอาด, ปี๊บ, ทราย, ใบเตย, เกลือป่น, น้ำปูนใส, น้ำส้มสายชู","ส้ม,กล้วย",59,กล้วยบวชชี,
7,ทาร์ตแอปเปิ้ล,"แป้งสาลีอเนกประสงค์, น้ำมันมะพร้าว, แอปเปิล","มะพร้าว,แอปเปิล",2431,ทาร์ตแอปเปิ้ล,
8,เอสเปรสโซ่มะม่วงปั่น (กาแฟน้ำมะม่วง),"กาแฟคั่วเอสเปรสโซ่, มะม่วงน้ำดอกไม้สุก, น้ำแข็ง, น้ำเปล่า","กาแฟ,มะม่วง",46,เอสเปรสโซ่มะม่วงปั่น (กาแฟน้ำมะม่วง),
9,ยำปูม้าแอปเปิลกีวี,"ปูม้า, แอปเปิล, กีวี","แอปเปิล,กีวี",1048,ยำปูม้าแอปเปิลกีวี,


In [10]:
#count fruits head
fcount=(table.set_index(['menu','ingredient','view','link','photo']).apply(lambda x: x.str.split(',').explode()).reset_index())
fcount['fruit'].value_counts().head(60)

กล้วย                165
เบอร์รี              119
มะนาว                98 
สตรอว์เบอร์รี        90 
มะม่วง               85 
กล้วยหอม             62 
ส้ม                  58 
แอปเปิล              54 
-                    53 
มะพร้าว              50 
สับปะรด              44 
ข้าวโพด              32 
กีวี                 30 
เสาวรส               28 
มะเขือเทศ            27 
ทุเรียน              19 
ถั่วลิสง             17 
แก้ว                 16 
แตงโม                16 
อัลมอนด์             16 
ฝรั่ง                15 
ท้อ                  15 
บลูเบอร์รี           14 
กระท้อน              13 
พีช                  13 
องุ่น                13 
มะขาม                13 
มะยงชิด              12 
โกโก้                12 
ส้มโอ                11 
ลำไย                 11 
เลมอน                11 
แก้วมังกร            11 
ตาล                  10 
กาแฟ                 9  
ฟักทอง               9  
มะม่วงหาวมะนาวโห่    8  
มัลเบอร์รี           8  
ลิ้นจี่              8  
ราสเบอร์รี           7  


In [11]:
#fruit count tail
fcount['fruit'].value_counts().tail(60)

กระท้อน              13
พีช                  13
องุ่น                13
มะขาม                13
มะยงชิด              12
โกโก้                12
ส้มโอ                11
ลำไย                 11
เลมอน                11
แก้วมังกร            11
ตาล                  10
กาแฟ                 9 
ฟักทอง               9 
มะม่วงหาวมะนาวโห่    8 
มัลเบอร์รี           8 
ลิ้นจี่              8 
ราสเบอร์รี           7 
บ๊วย                 7 
มะละกอ               7 
เงาะ                 6 
ถั่วเหลือง           5 
มะม่วงหิมพานต์       5 
ทับทิม               5 
ขนุน                 5 
จาก                  5 
สละ                  4 
มะกอก                4 
ฟักข้าว              4 
มังคุด               3 
อ้อย                 3 
เมล่อน               3 
บีทรูท               3 
ถั่วลันเตา           3 
อินทผลัม             3 
หม่อน                3 
ตะลิงปลิง            3 
กล้วยหักมุก          2 
มะปราง               2 
อะโวคาโด             2 
ส้มเขียวหวาน         2 
ทานตะวัน             2 
สาลี่           

In [12]:
#model of fruit recommendation
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import ipywidgets as widgets
from IPython.core.display import display, HTML, Markdown, clear_output
data = ['ลิ้นจี่', 'ลำไย', 'แอปเปิล', 'ทุเรียน','กล้วย','มะม่วง','มะพร้าว','เสาวรส','ทุเรียน','แตงโม','ข้าวโพด','มะนาว','ส้ม','สับปะรด','แอปเปิล','ฟักทอง','กระท้อน','มังคุด','สตรอว์เบอร์รี','-']
checkboxes = [widgets.Checkbox(value=False, description=label) for label in data]
vbox = widgets.VBox(children=checkboxes)
button=widgets.Button(description='find')
out = widgets.Output(layout={'border': '1px solid black'})

def find(b):
    selected_data = []
    for i in range(len(checkboxes)):
        if checkboxes[i].value == True:
            selected_data.append(checkboxes[i].description)
    f_input=selected_data
    f_test=table['fruit']
    s_menu=[]
    s_ing=[]
    s_fruit_have=[]
    s_view=[]
    s_link=[]
    s_img=[]
    header=[s_menu,s_ing,s_fruit_have,s_view,s_link,s_img]
    header_a=['menu','ingredient','fruit','view','link','photo']
    for i in range(len(table)):
        if fuzz.partial_ratio(f_test[i],f_input)>=70:
            for j in range(6):
                add=table[header_a[j]]
                header[j].append(add[i])
    s_table = pd.DataFrame({'menu': s_menu, 'ingredient': s_ing,'fruit': s_fruit_have,
                      'view': s_view, 'link':s_link, 'photo':s_img})
    table_rec=s_table.sort_values(['view'],ascending=False).reset_index()
    ttt = HTML(table_rec[['link','photo']].to_html(escape=False))
    if len(s_table)>0:
        with out:
            out.clear_output()
            #clear_output(True)
            display(ttt)
    else:
        out.clear_output()
        out.append_stdout('No recipe for you')

    
button.on_click(find)

display(vbox,button,out)

Button(description='find', style=ButtonStyle())

Output(layout=Layout(border='1px solid black'))